# Single Memristor Evolution and Form Factor

#### In this code we calculate the evolution of a quantum memristor given by:  
**Scientific Reports volume 7, Article number: 42044 (2017)**

#### and calculate the form factor of the resulting hysteresis curve.

We consider the single memristor model in superconducting circuits 
$$
\hat{H} = E_{C}\hat{n}^{2} + \frac{E_{L}}{2}(\hat{\varphi} - \hat{\varphi}_{d})^{2}
$$
where, $\hat{n}$ is the number of cooper pairs operator, $ \hat{\varphi}$ is the phase operator,  $E_{C} = 2e^{2}/C_{\Sigma}$ is the charge energy, with $C_{\Sigma} = C_{J_{1}} + C_{J_{2}}$, $E_{L} = \varphi_{0}^{2}/L$ is the inductive energy, and $\varphi_{d} = \phi_{d}/\varphi_{0}$ is the time-dependent external phase. 

In the Fock basis, this is

$$
H = \hbar \omega_{10} a^{\dagger}a
$$
with $\omega_{10} = \sqrt{2E_{C}E_{L}}/\hbar$

The system undergoes quasiparticle decay, described by the master equation
$$
\dot{\rho}(t) = -\frac{i}{\hbar}\big[\hat{H}, \rho \big] + \Gamma(t)\bigg(\hat{a}\rho \hat{a}^{\dagger} - \frac{1}{2}\{\hat{a}^{\dagger}\hat{a}, \rho \} \bigg)
$$
where $\Gamma(t) = g_{0}^{2}e^{-g_{0}^{2}} \bigg(\frac{1 + \cos(\phi_{d}(t)/2)}{2} \bigg)S_{\textrm{qp}}(\omega_{10})$, $g_{0} = (E_{C}/32E_{L})^{1/4}$ and $S_{\textrm{qp}}(\omega_{10}) = \omega_{10}$ is the spectral density. 

The quasiparticle current $\hat{I}_{\textrm{qp}}$ and the voltage across the capacitor $\hat{V}_{\textrm{cap}}$ are the quantities that show memristive dynamics, they can be obtained from the equations of motion of $\hat{n}$ and $\hat{\varphi}$. The memristive relation is given by
$$
\hat{I}_{\textrm{qp}} = G(t) \hat{V}_{\textrm{cap}}
$$
with $G(t) = \frac{g_{0}^{2}e^{-g_{0}^{2}}C_{\Sigma} S_{\textrm{qp}}(\omega)}{4} \bigg(1 + \cos\big(\frac{\phi_{d}(t)}{2}\big) \bigg)$, $\hat{V}_{\textrm{cap}} = -2e\langle \hat{n}\rangle/C_{\Sigma}$. Additionally we have
$$
\hat{n} = \frac{i}{4g_{0}}(a^{\dagger} - a) \\
\hat{\varphi} = 2g_{0}(a^{\dagger} + a)
$$


We calculate the form factor for the hysteresis curves that are drawn in the $\hat{I}_{\textrm{qp}}$-$\hat{V}_{\textrm{cap}}$ plot. The form factor involves calculating the area and perimeter of a closed figure, so it can only be calculated at certain times where hysteresis curve closes on itself. The form factor is given by 
$$
F = 4\pi\frac{A}{P^{2}}
$$


In [8]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.optimize as spo
import qutip as qp
import time
import pickle



## Function for memristor evolution

In [9]:
# =============================================================================
# Single Memristor evolution as a function
# =============================================================================

def Single_memristor_evolution(theta_1, theta_2, alpha):
    # =============================================================================
    # System Parameters
    # =============================================================================
    """ Fundamental constants"""
    hbar = 1.054571 * 1e-34  # Planck constant [J x s]
    phi0 = 2.067833 * 1e-15/(2*np.pi)  # Magnetic Flux quantum [Wb]
    G0 = 7.748091 * 1e-5  # Conductance quantum [Siemens]
    e = 1.602176 * 1e-19 # electron charge [C]
    
    "Circuit Parameters"
    # Based on the reference, the Capacitive energy is 1Ghz, and the
    # Inductive energy is 1000 times the capacitive energy. 
    # We choose the parameters corresponding to this condition
    Cg = 4.868269*1e-13  # Effective SQUID capacitance [Farad] => Cg = 2*(e**2)/(1e9*hbar)
    L = 1.027059*1e-9  # Inductance of outer loop [Henry]
    EC = 2 * (e**2)/Cg  # Charging energy in SQUID capacitor
    EL = (phi0**2)/L    # Inductive energy
    g0 = (EC/(32*EL))**(1/4) # Constant relevant for n and phi operators
    omega = np.sqrt(2*EC*EL)/(1e9*hbar) # Frequency of the harmonic oscillator, in units of GHz
    s_w = alpha * omega  # Spectral density  of environment
    nmax = 1  # Number of excitations in the harmonic oscillator
    
    # =============================================================================
    # Operators and decay
    # =============================================================================
    """ Define the operators and Hamiltonian of the system """
    a = qp.destroy(nmax+1)  # Annihilation operator
    I = qp.qeye(nmax+1)   # Identity
    H = omega*a.dag()*a   # Hamiltonian
 
    n = (1j/(4*g0))*(a.dag() - a) # Number of Cooper pairs operator
    phi = 2*g0*(a.dag() + a)   # Phase operator

    "Initial State"
    psi_0 = np.cos(theta_1/2)*qp.fock(nmax+1,1) + np.exp(1j*theta_2)*np.sin(theta_1/2) * qp.fock(nmax+1,0) 


    """ Define the time dependent decay rate """
    params = {'g0':g0}

    def gamma_t(t,args):
        g0 = args['g0']  # import g0 into function namespace
        phi_ini = np.pi/2  # phase shift of external flux
        phid = phi_ini + np.sin(omega*t)  # external flux
        gamma_0 = 0.25*g0**2 * np.exp(-g0**2) * s_w   # bare decay rate
        gamma_T = np.sqrt(gamma_0 * (1 + np.cos(phid)))  # We take square root because master equation solver squares the decay rate

        return gamma_T

    def gamma(t, args):
        g0 = args['g0']  # import g0 into function namespace
        phi_ini = np.pi/2  # phase shift of external flux
        phid = phi_ini + np.sin(omega*t)  # external flux
        gamma_0 = 0.25*g0**2 * np.exp(-g0**2) * s_w   # bare decay rate
        gamma = gamma_0 * (1 + np.cos(phid))  # We take square root because master equation solver squares the decay rate

        return gamma

    "Collapse operators"
    c_ops = [[a, gamma_t]]  # time-dependent collapse term

    # =============================================================================
    # System Dynamics
    # =============================================================================
    """ Dynamics parameters """
    number_oscillations = 5
    points_per_oscillation = 500
    time_points = number_oscillations * points_per_oscillation
    timescale = 2*np.pi/(omega)  # Period of the oscillator
    t = timescale * np.linspace(0, number_oscillations, time_points) # Time vector for our time evolution

    rho_t = qp.mesolve(H, psi_0, t, c_ops, args=params)  # Solve master equation

    V = np.zeros(len(t))  # Voltage vector
    P = np.zeros(len(t))  # Phase vector
    Iqp = np.zeros(len(t))  # Quasiparticle current vector

    for i in range(len(t)):
        rho_ti = rho_t.states[i]
        V[i] = -2*e*qp.expect(n,rho_ti)/Cg    # Voltage
        P[i] = qp.expect(phi,rho_ti)   # Phase
        Iqp[i] = gamma(t[i], params)*V[i]*Cg   # Quasiparticle Current
        
    """ Define maximum initial voltage and quasiparticle current """
    psi_max_volt = np.cos(np.pi/4)*qp.fock(nmax+1,1) + np.exp(1j*np.pi/2)*np.sin(np.pi/4) * qp.fock(nmax+1,0) 
    
    V0 = -2*e*qp.expect(n,psi_max_volt)/Cg  # Normalizing voltage constant
    I0 = V0 * 0.25* g0**2 * np.exp(-g0**2)* (100*omega) * Cg     # Normalizing quasiparticle current constant considering alpha = 100
    
    # =============================================================================
    # Auxiliary functions
    # =============================================================================

    def Identify_loops(x,y):
        """
        This function stores the indices of the point in time
        when x crosses zero value, and stores them in the indices
        container.

        One memristive hysteresis curve crosses the origin two times during its entire loop
        So the indices that denote the starting point of each hysteresis curve are obtained
        by taking every other element from indices container.
        This is stored in the real_indices container.
        """
        indices = []
        real_indices = []
        for ind in range(len(x)-1):
            prod_sign = np.sign(x[ind]*x[ind+1])
            if prod_sign == -1:
                closest_ind = np.argmin(np.abs([x[ind], x[ind+1]])) # Choose which point is closer to zero n or n+1
                if closest_ind == 0:
                    indices.append(ind)
                else:
                    indices.append(ind+1)

        for n in range(int(len(indices)/2) ):
            real_indices.append(indices[2*n])  
        return real_indices  # 

    def Area(x,y):
        "Area is calculated with Green's theorem"
        A = 0
        for i in range(len(x)-1):
            A += 0.5*abs(y[i]*(x[i+1]-x[i]) - x[i]*(y[i+1]-y[i]))
        return A

    def Perimeter(x,y):
        L = 0
        for i in range(len(x)-1):
            L += np.sqrt((x[i+1] - x[i])**2 + (y[i+1] - y[i])**2)
        return L

    # =============================================================================
    # Form Factor
    # =============================================================================
    def Form_factor(x,y):
        indices = Identify_loops(x,y)
        number_of_loops = len(indices)-1
        form_factor = np.zeros([number_of_loops])

        for n in range(number_of_loops):
            loop_x = x[indices[n]:indices[n+1]]
            loop_y = y[indices[n]:indices[n+1]]
            form_factor[n] = 4*np.pi*Area(loop_x,loop_y)/(Perimeter(loop_x,loop_y)**2)

        return form_factor


    # =============================================================================
    # Calculate Form Factor for current Dynamics
    # =============================================================================

    FF = Form_factor(V/V0,Iqp/I0) # Form factor for normalized current and voltage

    indices = Identify_loops(V/V0,Iqp/I0)  # indices of points in time when a full loop is completed
    
    parameters = {'omega':omega, 'V0':V0, 'I0':I0, 'timescale':timescale} 
    
    return FF[0], V, Iqp, t, indices, parameters

## Calculate the Data

In [10]:

"""  Controllable Parameters  """

"We parametrize inital state as = cos(theta_1/2)|0> + e^(i*theta_2)sin(theta_1/2)|1>"
theta_1 = np.pi/2  # Fix theta_1 to maximize initial voltage

"Range of values for theta_2 and alpha"
theta2_min = 0.01
theta2_max = 0.99*2*np.pi

alpha_min = 0.1
alpha_max = 100


"Preallocation"
n_samples = 2000  # Number of random samples to consider

# We store the relevant data into a structured array
Data_dtype = np.dtype([('theta_2', np.float32), 
                       ('alpha', np.float32),
                       ('Formfactor', np.float32)])

Data = np.zeros(n_samples, dtype = Data_dtype)

V = []
Iqp = []

for k1 in range(n_samples):
    theta_2 = np.random.rand()*(theta2_max - theta2_min) + theta2_min  # random theta_2 value
    alpha = np.random.rand()*(alpha_max - alpha_min) + alpha_min  # random alpha value
    
    FF, V_aux, Iqp_aux, t, indices, parameters = Single_memristor_evolution(theta_1 = theta_1, theta_2 = theta_2, alpha = alpha)
    Data['theta_2'][k1] = theta_2
    Data['alpha'][k1] = alpha
    Data['Formfactor'][k1] = FF
    V.append(V_aux)
    Iqp.append(Iqp_aux)
    time = t
        

    print('k1 = ', k1)
    
omega = parameters['omega']
I0 = parameters['I0']
V0 = parameters['V0']
timescale = parameters['timescale']



    
    


k1 =  0
k1 =  1
k1 =  2
k1 =  3
k1 =  4
k1 =  5
k1 =  6
k1 =  7
k1 =  8
k1 =  9
k1 =  10
k1 =  11
k1 =  12
k1 =  13
k1 =  14
k1 =  15
k1 =  16
k1 =  17
k1 =  18
k1 =  19
k1 =  20
k1 =  21
k1 =  22
k1 =  23
k1 =  24
k1 =  25
k1 =  26
k1 =  27
k1 =  28
k1 =  29
k1 =  30
k1 =  31
k1 =  32
k1 =  33
k1 =  34
k1 =  35
k1 =  36
k1 =  37
k1 =  38
k1 =  39
k1 =  40
k1 =  41
k1 =  42
k1 =  43
k1 =  44
k1 =  45
k1 =  46
k1 =  47
k1 =  48
k1 =  49
k1 =  50
k1 =  51
k1 =  52
k1 =  53
k1 =  54
k1 =  55
k1 =  56
k1 =  57
k1 =  58
k1 =  59
k1 =  60
k1 =  61
k1 =  62
k1 =  63
k1 =  64
k1 =  65
k1 =  66
k1 =  67
k1 =  68
k1 =  69
k1 =  70
k1 =  71
k1 =  72
k1 =  73
k1 =  74
k1 =  75
k1 =  76
k1 =  77
k1 =  78
k1 =  79
k1 =  80
k1 =  81
k1 =  82
k1 =  83
k1 =  84
k1 =  85
k1 =  86
k1 =  87
k1 =  88
k1 =  89
k1 =  90
k1 =  91
k1 =  92
k1 =  93
k1 =  94
k1 =  95
k1 =  96
k1 =  97
k1 =  98
k1 =  99
k1 =  100
k1 =  101
k1 =  102
k1 =  103
k1 =  104
k1 =  105
k1 =  106
k1 =  107
k1 =  108
k1 =  109
k1 =  110


# Save Data

In [5]:
Info = ['We calculate the form factor for a single memristor for 2000 random values of theta_2 and alpha',
        'Data is stored in "Data" variable as a structured array as [theta_2, alpha, formfactor]',
        'Additionally we store a list of Voltage, V, and quasiparticle current, Iqp as a function of time',
        'Each of these 2 lists contain 2000 elements, each element is an array with the values of V and Iqp in time',
        'the "time" variable is the time series array corresponding to the evolution, it is the same for all cases'
        'Finally the parameters variable, stores additional parameters that may be important',
        'These are omega, I0, V0, timescale'
       ]

# with open('Data_single_memristor/Single_memristor_formfactor.dat', 'wb') as arg:
#     pickle.dump([Data, V, Iqp, time, parameters, Info], arg)